In [ ]:
#!pip install -U albumentations
#!pip install -U ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
# Carrega o nome das classes
with open(r'C:\YOLO\classes.txt', 'r') as f:
    class_names = f.read().splitlines()
    
    print(class_names)

In [ ]:
# Cria o data.yaml com as informações do conjunto de dados
data_yaml = f"""
path: dataset  # caminho para o conjunto de dados
train: images/train  # diretório de imagens de treinamento
val: images/test  # diretório de imagens de teste

nc: {len(class_names)}  # número de classes
names: {class_names}  # nomes das classes
"""

with open('./model/data.yaml', 'w') as f:
    f.write(data_yaml)

In [ ]:
# Carrega o modelo YOLO
model = YOLO("./model/yolo11n.pt")

In [ ]:
# Parâmetros do modelo que serão otimizados no tuning
search_space = {
    "lr0": (1e-5, 1e-1),           # Taxa de aprendizado inicial
    "lrf": (0.01, 1.0),            # Fator final da taxa de aprendizado
    "momentum": (0.6, 0.98),       # Fator de momentum do SGD
    "weight_decay": (0.0, 0.001),    # Regularização L2 para evitar overfitting
    "warmup_epochs": (0.0, 5.0),     # Número de épocas para warmup
    "warmup_momentum": (0.0, 0.95),  # Momentum inicial durante o warmup
    "box": (0.02, 0.2),            # Peso da perda de caixa (bounding box)
    "cls": (0.2, 4.0),             # Peso da perda de classificação
    "hsv_h": (0.0, 0.1),           # Alcance de variação de hue (matiz)
    "hsv_s": (0.0, 0.9),           # Alcance de saturação
    "hsv_v": (0.0, 0.9),           # Alcance de variação de valor (brilho)
    "degrees": (0.0, 45.0),        # Rotação máxima em graus para augmentations
    "translate": (0.0, 0.9),       # Translação máxima como fração do tamanho da imagem
    "scale": (0.0, 0.9),           # Escala para augmentations
    "shear": (0.0, 10.0),          # Cisalhamento máximo em graus
    "perspective": (0.0, 0.001),   # Perspectiva para augmentations
    "flipud": (0.0, 1.0),          # Probabilidade de flip vertical
    "fliplr": (0.0, 1.0),          # Probabilidade de flip horizontal
    "mosaic": (0.0, 1.0),          # Probabilidade de usar mosaic augmentation
    "mixup": (0.0, 1.0),           # Probabilidade de usar mixup augmentation
    "copy_paste": (0.0, 1.0)       # Probabilidade de usar copy-paste augmentation
}

In [ ]:
# Inicia o tuning
model.tune(
    data="./model/data.yaml",
    epochs=30,
    iterations=50,
    optimizer="SGD",
    batch=50,
    space=search_space,
    plots=False,
    save=False,
    val=True,
    project='./runs/detect/tuning/SGD',
    cos_lr=True,
    amp=True,
    deterministic=True
)